In [3]:
from nexus.nexus_api import API
from nexus.utils.time_point import TEMPORAL_GRANU
from nexus.utils.coordinate import SPATIAL_GRANU
import os
os.environ["CONFIG_FILE_PATH"] = "config_test.yaml" 
os.chdir(f"/Users/yuegong/nexus_correlation_discovery")

In [4]:
conn_str = f'data/data_commons.db'
nexus_api = API(conn_str, data_sources=['data_commons'])

config_test.yaml


In [5]:
# test find correlations from a single table
dataset = '00021_Social_Chicago'
# asthma data only has spatial attribute, thus the temporal granularity is set to ALL.
temporal_granularity, spatial_granularity = TEMPORAL_GRANU.ALL, SPATIAL_GRANU.TRACT
overlap_threshold = 100
correlation_threshold = 0.5
# you can change correlation_type to 'spearman' or 'kendall'
correlations = nexus_api.find_correlations_from(dataset, temporal_granularity, spatial_granularity, 
                                      overlap_threshold, correlation_threshold, 
                                      correlation_type="pearson")

config_test.yaml
total number of correlations: 907


In [6]:
# test finding all correlations
from nexus.data_search.commons import FIND_JOIN_METHOD

nexus_api.data_sources = ['data_commons']
temporal_granularity, spatial_granularity = None, SPATIAL_GRANU.TRACT
overlap_threshold = 100
correlation_threshold = 0.5
persist_path = 'tmp/test/'
all_correlations = nexus_api.find_all_correlations(temporal_granularity, spatial_granularity,
                                        overlap_threshold, correlation_threshold,
                                        persist_path=persist_path, correlation_type="pearson",
                                        find_join_method=FIND_JOIN_METHOD.JOIN_ALL)
print(len(all_correlations))

  0%|          | 0/17 [00:00<?, ?it/s]

 12%|█▏        | 2/17 [00:00<00:01, 10.80it/s]

skip because this table does not have enough keys


100%|██████████| 17/17 [00:08<00:00,  1.99it/s]


total number of correlations: 8060
8060


# Can LLM help us interpret correlations?

Test correlation 1: Attribute ViolentRate1000 from table 00021_Social_Chicago is correlated with Attribute pct_Civ_unemp_16p_ACS from table 00011_ACS1519_Chicago at the census tract level. The correlation coefficient is 0.66.

Ground truth explanation: The violent crime rate is positively correlated with the percentage of civilians ages 16 years and over in the labor force that is unemployed.

Test correlation 2: Attribute ViolentRate1000 from table 00021_Social_Chicago is correlated with Attribute pct_MrdCple_HHD from table 00011_ACS1519_Chicago at the census tract level. The correlation coefficient is -0.64.

Ground truth explanation: The violent crime rate is negatively correlated with the percentage of Married-Couple Households.

In [5]:
import ollama

class ChatBot:
    def __init__(self, model: str):
        self.model = model
        self.message_queue = []

    def generate(self, msg: str, options=None):
        """
        in the mode of generation, chat without history
        """
        stream = ollama.generate(model=self.model, 
            prompt=msg,
            stream=True,
            options=options
          )
    
        response = ""
        for chunk in stream:
            part = chunk['response']
            print(part, end='', flush=True)
            response = response + part
        print(response)
    
    def chat(self, msg: str, options=None):
      self.message_queue.append(
        {
          'role': 'user',
          'content': msg,
        }
      )
      stream = ollama.chat(model=self.model, 
        messages=self.message_queue,
        stream=True,
        options=options
      )
    
      response = ""
      for chunk in stream:
        part = chunk['message']['content']
        print(part, end='', flush=True)
        response = response + part
    
      self.message_queue.append(
        {
          'role': 'assistant',
          'content': response,
        }
      )
    
      print(response)

    def clear_message_queue():
        self.message_queue.clear()

## The ability to decipher attribute names

In [6]:
client = ChatBot('tinyllama')

In [11]:
question = """

Attribute "ViolentRate1000" in the table "00021_Social_Chicago" means Violent rate * 1000.

What does the attribute "pct_Civ_unemp_16p_ACS" mean in a table "00011_ACS1519_Chicago"?

Write your response in the format of "pct_Civ_unemp_16p_ACS: your answer"

"""
# todo: output format?

client.generate(msg=question, options={"seed": 123, "temperature": 0})

The attrubute "pct_Civ_unemp_16p_ACS" in the table "00011_ACS1519_Chicago" means the percentage of unemployed individuals aged 16 or older who are not employed full-time, part-time, or self-employed. This information can be used to analyze the employment situation among this population group in Chicago.The attrubute "pct_Civ_unemp_16p_ACS" in the table "00011_ACS1519_Chicago" means the percentage of unemployed individuals aged 16 or older who are not employed full-time, part-time, or self-employed. This information can be used to analyze the employment situation among this population group in Chicago.


In [8]:
question = "attribute ViolentRate1000 from table 00021_Social_Chicago is correlated with \
            attribute pct_Civ_unemp_16p_ACS from table 00011_ACS1519_Chicago at the census tract level.\
            The correlation coefficent is 0.66. Could you please describe this correlation using easily understandable language,\
            with all attribute names clarified?"
client.chat(msg=question, options={"seed": 123, "temperature": 0})

Certainly! Here's a simple explanation of the correlation between ViolentRate1000 and pcT_Civ_unemp_16p_ACS using easily understandable language:

ViolentRate1000 is an attribute from table 00021_Social_Chicago, which corresponds to the "Violent Rate (1000)" metric in the Census Bureau's Public Consulting Group (PCG) Table 00011_ACS1519_Chicago. The correlation coefficient between ViolentRate1000 and pcT_Civ_unemp_16p_ACS is 0.66, which indicates a strong positive correlation between the two attributes. This means that for every increase in ViolentRate1000 there is a corresponding increase in the number of people living in poverty who are unemployed and not working full-time (i.e., those with jobs but no income).Certainly! Here's a simple explanation of the correlation between ViolentRate1000 and pcT_Civ_unemp_16p_ACS using easily understandable language:

ViolentRate1000 is an attribute from table 00021_Social_Chicago, which corresponds to the "Violent Rate (1000)" metric in the Censu

In [10]:
question = "do you find this correlation intriguing and worth further investigation?"
client.chat(msg=question, options={"seed": 123, "temperature": 0})

I do not have the ability to conduct further research or make personal judgments. However, based on the provided information, the correlation between viololenrtate1000 and pcT_civ_unemp_16p_acs is of interest and may be worth further investigation. The data presented in the table and census tract level analysis suggest that there is a positive correlation between these two attributes, which could have implications for poverty levels and job opportunities in chicago. Further research into this topic would likely provide more insights into how these two metrics are related and how they can be used to address poverty and job opportunities in the city.I do not have the ability to conduct further research or make personal judgments. However, based on the provided information, the correlation between viololenrtate1000 and pcT_civ_unemp_16p_acs is of interest and may be worth further investigation. The data presented in the table and census tract level analysis suggest that there is a positiv